# SQL Subqueries & Temporary tables

## SubQueries
Can be used anywhere you can use a table name.  
If the return of the subquery is one result then it can be used in **WHERE**, **HAVING**, SELECT in the **CASE** statement.

Example:
```sql
SELECT channel, AVG(event_count) avg_event_count
FROM
(SELECT DATE_TRUNC('day', occurred_at) day, channel, COUNT(*) event_count
FROM web_events
GROUP BY 1, 2
) sub
GROUP BY 1
ORDER BY 2 DESC
```

Questions:
1. Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
```sql
SELECT t3.rep_name, t3.region_name, t3.total_amt
FROM(SELECT region_name, MAX(total_amt) total_amt
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
             FROM sales_reps s
             JOIN accounts a
             ON a.sales_rep_id = s.id
             JOIN orders o
             ON o.account_id = a.id
             JOIN region r
             ON r.id = s.region_id
             GROUP BY 1, 2) t1
     GROUP BY 1) t2
JOIN (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
     FROM sales_reps s
     JOIN accounts a
     ON a.sales_rep_id = s.id
     JOIN orders o
     ON o.account_id = a.id
     JOIN region r
     ON r.id = s.region_id
     GROUP BY 1,2
     ORDER BY 3 DESC) t3
ON t3.region_name = t2.region_name AND t3.total_amt = t2.total_amt;
```
2. For the region with the largest (sum) of sales total_amt_usd, how many total (count) orders were placed?
```sql
SELECT r.name, COUNT(o.total) total_orders
FROM orders o
JOIN accounts a ON o.account_id = a.id
JOIN sales_reps s ON a.sales_rep_id = s.id
JOIN region r ON s.region_id = r.id
GROUP BY r.name
HAVING SUM(o.total_amt_usd) = (SELECT MAX(total_amount)
FROM(SELECT r.name region_name, SUM(o.total_amt_usd) total_amount
FROM orders o
JOIN accounts a ON o.account_id = a.id
JOIN sales_reps s ON a.sales_rep_id = s.id
JOIN region r ON s.region_id = r.id
GROUP BY region_name) sub)
```
3. How many accounts had more total purchases ,than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?
```sql
SELECT COUNT(*)
FROM (SELECT a.name
FROM orders o
JOIN accounts a ON o.account_id = a.id
GROUP BY 1
HAVING SUM(o.total) > (SELECT total
FROM (SELECT a.name account_name, SUM(o.standard_qty) as total_qty, SUM (o.total) total
FROM orders o
JOIN accounts a ON o.account_id = a.id
GROUP BY a.name
ORDER BY total_qty DESC
LIMIT 1) sub))counter_tab;
```
4. For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?
```sql
SELECT a.name, w.channel, COUNT(*)
FROM accounts a
JOIN web_events w
ON a.id = w.account_id AND a.id =  (SELECT id
                     FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
                           FROM orders o
                           JOIN accounts a
                           ON a.id = o.account_id
                           GROUP BY a.id, a.name
                           ORDER BY 3 DESC
                           LIMIT 1) inner_table)
GROUP BY 1, 2
ORDER BY 3 DESC;
```
5. What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?
```sql
SELECT AVG(total_spending)
FROM (SELECT a.name account_name, SUM(o.total_amt_usd) total_spending, COUNT(o.*) num_orders
FROM orders o
JOIN accounts a ON o.account_id = a.id
GROUP BY a.name
ORDER BY total_spending DESC
LIMIT 10) t1
```
6. What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.
```sql
SELECT AVG(order_avg)
FROM (SELECT AVG(o.total_amt_usd) order_avg
FROM orders o
GROUP BY o.account_id
HAVING AVG(o.total_amt_usd) > (SELECT AVG(o.total_amt_usd)
FROM orders o)
ORDER BY order_avg) t
```



## WITH statement
Serving the same purpose of subqueries they are more common as they tend to be cleaner to read.

**CTE** - common table expression

Example:
```sql
SELECT channel, AVG(event_count) AS avg_event_count
FROM (
    SELECT DATE_TRUNC('day', occurred_at) AS day, channel, COUNT(*) AS event_count
    FROM web_events
    GROUP BY 1,2
) sub
GROUP BY 1
ORDER BY 2 DESC
```
Becomes:
```sql
WITH events AS (
    SELECT DATE_TRUNC('day', occurred_at) AS day, channel, COUNT(*) AS event_count
    FROM web_events
    GROUP BY 1,2
    )

SELECT channel, AVG(event_count) AS avg_event_count
FROM events
GROUP BY 1
ORDER BY 2 DESC
```
Test Questions:
1. Provide the name of the sales_rep in each region with the largest amount of total_amt_usd sales.
```sql
WITH t1 AS (
        SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
        FROM sales_reps s
        JOIN accounts a
        ON a.sales_rep_id = s.id
        JOIN orders o
        ON o.account_id = a.id
        JOIN region r
        ON r.id = s.region_id
        GROUP BY 1, 2),
t2 AS (
        SELECT region_name, MAX(total_amt) total_amt
        FROM t1
        GROUP BY 1),
t3 AS (
        SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt
        FROM sales_reps s
        JOIN accounts a
        ON a.sales_rep_id = s.id
        JOIN orders o
        ON o.account_id = a.id
        JOIN region r
        ON r.id = s.region_id
        GROUP BY 1,2
        ORDER BY 3 DESC)

SELECT t3.rep_name, t3.region_name, t3.total_amt
FROM t2
JOIN t3
ON t3.region_name = t2.region_name AND t3.total_amt = t2.total_amt;
```
2. For the region with the largest sales total_amt_usd, how many total orders were placed?
```sql
WITH sub AS (
    SELECT r.name region_name, SUM(o.total_amt_usd) total_amount
    FROM orders o
    JOIN accounts a ON o.account_id = a.id
    JOIN sales_reps s ON a.sales_rep_id = s.id
    JOIN region r ON s.region_id = r.id
    GROUP BY region_name),
    maxa AS (
    SELECT MAX(total_amount)
    FROM sub)

SELECT r.name, COUNT(o.total) total_orders
FROM orders o
JOIN accounts a ON o.account_id = a.id
JOIN sales_reps s ON a.sales_rep_id = s.id
JOIN region r ON s.region_id = r.id
GROUP BY r.name
HAVING SUM(o.total_amt_usd) = (SELECT * FROM maxa)
```
3. How many accounts had more total purchases than the account name which has bought the most standard_qty paper throughout their lifetime as a customer?
```sql
WITH sub AS (
    SELECT a.name account_name, SUM(o.standard_qty) as total_qty, SUM (o.total) total
    FROM orders o
    JOIN accounts a ON o.account_id = a.id
    GROUP BY a.name
    ORDER BY total_qty DESC
    LIMIT 1),
    counter_tab AS (
    SELECT a.name
    FROM orders o
    JOIN accounts a ON o.account_id = a.id
    GROUP BY 1
    HAVING SUM(o.total) > (SELECT total
    FROM sub))

SELECT COUNT(*)
FROM counter_tab;
```
4. For the customer that spent the most (in total over their lifetime as a customer) total_amt_usd, how many web_events did they have for each channel?
```sql
WITH inner_table AS (
    SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent
    FROM orders o
    JOIN accounts a
    ON a.id = o.account_id
    GROUP BY a.id, a.name
    ORDER BY 3 DESC
    LIMIT 1)

SELECT a.name, w.channel, COUNT(*)
FROM accounts a
JOIN web_events w
ON a.id = w.account_id AND a.id =  (
SELECT id
FROM  inner_table)
GROUP BY 1, 2
ORDER BY 3 DESC;
```
5. What is the lifetime average amount spent in terms of total_amt_usd for the top 10 total spending accounts?
```sql
WITH t1 AS (
    SELECT a.name account_name, SUM(o.total_amt_usd) total_spending, COUNT(o.*) num_orders
    FROM orders o
    JOIN accounts a ON o.account_id = a.id
    GROUP BY a.name
    ORDER BY total_spending DESC
    LIMIT 10)

SELECT AVG(total_spending)
FROM  t1
```
6. What is the lifetime average amount spent in terms of total_amt_usd, including only the companies that spent more per order, on average, than the average of all orders.
```sql
WITH t AS (
    SELECT AVG(o.total_amt_usd) order_avg
    FROM orders o
    GROUP BY o.account_id
    HAVING AVG(o.total_amt_usd) > (SELECT AVG(o.total_amt_usd)
    FROM orders o)
    ORDER BY order_avg)

SELECT AVG(order_avg)
FROM  t
```

